In [14]:
# Add song embeddings as node features 


In [49]:
import torch
from dataset import Dataset
dataset = Dataset()
dataset.load("data")
print(len(dataset.nodes))
print(len(dataset.edges))

299
412


In [50]:
count = 0
for i in dataset.nodes:
    if "song_urls" in i[1].keys():
        count += 1
count / len(dataset.nodes)

1.0

In [51]:
# Remove edges with no ids in the nodes (the collection was stopped before expansion to these nodes)
edge_ids = [i[0][0] for i in dataset.edges] + [i[0][1] for i in dataset.edges]
edge_ids = list(set(edge_ids))
node_ids = [i[0] for i in dataset.nodes]

not_in_1 = [i for i in edge_ids if i not in node_ids]
not_in_2 = [i for i in node_ids if i not in edge_ids]

print(len(not_in_1))
print(len(not_in_2))

print(len(dataset.edges))
dataset.edges = [i for i in dataset.edges if i[0][1] not in not_in_1]
print(len(dataset.edges))


23
0
412
348


In [52]:
# Change the idx of the album nodes
id_map = {}
for idx, node in enumerate(dataset.nodes):
    id_map[node[0]] = idx
    node[0] = idx

# And change their indices in the album_artist_edges as well
for edge in dataset.edges:
    edge[0][0] = id_map[edge[0][0]]
    edge[0][1] = id_map[edge[0][1]]


In [53]:
x = torch.Tensor([i[1]["popularity"] for i in dataset.nodes]).t().contiguous().long()
edges = torch.Tensor([[i[0][0], i[0][1]] for i in dataset.edges]).t().contiguous().long()

print(x.shape, x.dtype)
print(edges.shape, edges.dtype)


torch.Size([299]) torch.int64
torch.Size([2, 348]) torch.int64


In [54]:
from torch_geometric.data import Data

data = Data(x=x, edge_index=edges)
 
print(">>> num_nodes:", data.num_nodes)
print(">>> num_edges:", data.num_edges)
data.validate()


>>> num_nodes: 299
>>> num_edges: 348


True

In [55]:
from torch_geometric.loader import NeighborLoader
sampler = NeighborLoader(data, num_neighbors=[2]*2, batch_size=16)


In [ ]:
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

samples = [i for i in iter(sampler)]
print(">>> n_samples:", len(samples))

random_idx = torch.randint(0, len(samples), (1,)).item()
random_idx = 0
print(">>> random_idx:", random_idx)
sample = samples[random_idx]
print(sample.keys)

print(sample.x.shape)
print(sample.edge_index.shape)


In [59]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
import networkx

G = to_networkx(data)
plot = figure(tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
              x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), width=800)
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))
network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
plot.renderers.append(network_graph)
output_notebook(hide_banner=True)
show(plot)

In [82]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Batch

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(5, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.sigmoid(x)
        return x

model = GCN(hidden_channels=10).cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

samples = [i.cuda() for i in iter(sampler)]
batches = [i.to_homogeneous() for i in iter(sampler)]

for idx, batch in enumerate(batches):
    optimizer.zero_grad()

    print(type(batch))
    print(dir(batch))

    print(batch[0])

    stop

    print(sample.x.shape)
    # x = sample.x[:5].unsqueeze(0)
    # y = sample.y[0]
    # edge_index = sample.edge_index

    # out = model(x, torch.empty(2, 0, dtype=torch.long).cuda())
    # loss = criterion(out.squeeze(), y.squeeze(0))

    # print(loss.item(), out.item(), y.item())



<class 'torch_geometric.data.data.Data'>
['__call__', '__cat_dim__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__inc__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_edge_attr_cls', '_edge_to_layout', '_edges_to_layout', '_get_edge_index', '_get_tensor', '_get_tensor_size', '_multi_get_tensor', '_put_edge_index', '_put_tensor', '_remove_edge_index', '_remove_tensor', '_store', '_tensor_attr_cls', '_to_type', 'apply', 'apply_', 'batch', 'clone', 'coalesce', 'contains_isolated_nodes', 'contains_self_loops', 'contiguous', 'coo', 'cpu', 'csc', 'csr', 'cuda', 'debug', 'detach', '

KeyError: 0